In [1]:
import os
os.chdir("..")

In [2]:
import numpy as np
import pickle
import json
import time
import random

import osmnx as ox

from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib import colormaps as cmaps
from matplotlib import cm

from data import get_data
from random_walks import get_end_nodes

In [3]:
with open("./places.json") as fin:
    places = json.load(fin)

## Objectives

Generate SVG plots of:
1. Every place and boba shops within them.
2. 10 random walks within each place.

In [4]:
def plot_expected_time(slug, data, cmap="jet"):
    """
    data is G, adj, boba_gdf.

    Returns fig, ax of the plot.
    Saves the plot to svg?
    """
    G, adj, boba_gdf = data
    with open(f"./output/avg_time/{slug}.pkl", "rb") as fin:
        E_time = pickle.load(fin)[(60, 5)]
    
    node2idx = {node: idx for idx, node in enumerate(sorted(G.nodes()))}

    print(f"  Calculating colors...")
    times = np.array([E_time[node2idx[node]] for node in G.nodes()]) / 60
    q1 = 0 #np.quantile(times, 0.1)
    q3 = 60 #np.quantile(times, 0.9)
    node_colors = cmaps[cmap]((times - q1) / (q3 - q1))
    
    print(f"  Generating plot...")
    fig, ax = ox.plot_graph(
        G, figsize=(8, 8),
        bgcolor="#fff",
        node_size=5, edge_linewidth=0.5,
        node_color=node_colors, edge_color="#444",
        show=False, close=True
    )
    
    # Make the colorbar
    norm = plt.Normalize(vmin=q1, vmax=q3)
    cb = fig.colorbar(
        cm.ScalarMappable(norm=norm, cmap=cmap),
        ax=ax, orientation="vertical",
        fraction=0.04, pad=0.04
    )
    cb.set_label("Expected time to reach boba by random walk (hours)")

    return fig, ax

In [11]:
slugs = ["harvard", "uw_seattle", "national_taiwan_university", "stanford"]

for placename, coords, slug in places:
    if slug not in slugs: continue

    start_time = time.time()
    print(f"Generating plots for {placename}...")
    
    data = get_data(slug)
    fig, ax = plot_expected_time(slug, data)
    print(f"  Done in {(time.time() - start_time):.3f} sec.")

    os.makedirs("./output/figures", exist_ok=True)
    fig.savefig(f"./output/figures/exp_time_{slug}.png", bbox_inches="tight")

Generating plots for Harvard...
  Calculating colors...
  Generating plot...
  Done in 7.359 sec.
Generating plots for UW Seattle...
  Calculating colors...
  Generating plot...
  Done in 8.320 sec.
Generating plots for Stanford...
  Calculating colors...
  Generating plot...
  Done in 0.940 sec.
Generating plots for National Taiwan University...
  Calculating colors...
  Generating plot...
  Done in 1.394 sec.


## Plot boba locations and sample random walks

In [8]:
from pprint import pprint

import random_walks
random_walk = random_walks.random_walk

In [19]:
def plot_boba_walks(data):
    """
    Plot the points in a city, and a few routes.
    """
    G, adj, boba_gdf = data
    end_nodes = get_end_nodes(G, boba_gdf)

    plot_scale = 8
    fig, ax = ox.plot_graph(
        G, bgcolor="#fff",
        # dpi=2000,
        # figsize=(8, 8),
        figsize=(8 * plot_scale,) * 2,
        node_size=1 * plot_scale**2, edge_linewidth=0,
        node_color="#80808080",
        show=False, close=True
    )
    for _, row in boba_gdf.iterrows():
        x, y = row.geometry.x, row.geometry.y
        ax.plot(x, y, "b*", markersize=8 * plot_scale, alpha=0.5)

    starts = random.choices(list(G.nodes()), k=3)
    for start in starts:
        route, total_time, flag = random_walk(
            adj, end_nodes, start,
            angle_cutoff=np.pi/3, forward_weight=5,
            timeout=np.inf
        )
        points = np.array([(G.nodes[p]["x"], G.nodes[p]["y"]) for p in route])
        ax.plot(*points.T, lw=2 * plot_scale, alpha=0.5)
    
    return fig, ax

In [20]:
for slug in ["harvard", "uw_seattle", "national_taiwan_university", "stanford"]:
    print(f"Generating random walks for slug {slug}...")
    data = get_data(slug)
    fig, ax = plot_boba_walks(data)
    fig.savefig(f"./output/figures/sample_walks_{slug}.png", bbox_inches="tight")

Generating random walks for slug harvard...
Generating random walks for slug uw_seattle...
Generating random walks for slug national_taiwan_university...
Generating random walks for slug stanford...
